In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# --- Parâmetros ---
VIDEO_PATH = '/content/20250520_084850.jpg'  # Troque para 0 para webcam, ou caminho do vídeo
REAL_MARKER_SIZE = 0.05
DISTANCE_THRESHOLD = 2.0

# calibragem da câmera
camera_matrix = np.array([[1000, 0, 640],
                          [0, 1000, 480],
                          [0, 0, 1]], dtype=np.float32)

dist_coeffs = np.zeros((4, 1), dtype=np.float32)

# --- Inicializar detector ArUco (dicionário padrão 6x6_250) ---
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

cap = cv2.VideoCapture(VIDEO_PATH)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    # --- Processar marcadores detectados ---
    if ids is not None:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, REAL_MARKER_SIZE, camera_matrix, dist_coeffs)

        filtered_ids = []
        for i in range(len(ids)):
            distance = tvecs[i][0][2]

            if distance <= DISTANCE_THRESHOLD:
                filtered_ids.append(ids[i][0])

                corner_center = corners[i][0].mean(axis=0).astype(int)
                text_pos = (corner_center[0], corner_center[1] + 20)

                distance_text = f"ID: {ids[i][0]} Dist: {distance:.2f}m"

                cv2.putText(frame, distance_text, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


        if filtered_ids:
             print(f"IDs dentro de {DISTANCE_THRESHOLD}m: {filtered_ids}")
             if 1 in filtered_ids:
                 print("LED ON para marcador 1 (dentro do limite de distância)!")
        else:
             print(f"Nenhum marcador detectado dentro de {DISTANCE_THRESHOLD}m.")


    cv2_imshow(frame)

cap.release()
cv2.destroyAllWindows()